How to use splotch to plot one image per condition

beta_level_1 used in post-processing of splotch output
 /ahg/regevdata/projects/ST_SpatialTranscriptomics/splotch/scripts/assist/read_splotch_newOuputFolder.py used on cluster
 splotch_two_levels
 12 warmup samples
 same order of columns in count file and annotation file, used in splotch
 called /outputs/ST_43/splotch/output_v4 on the server

In [1]:
import os
import re
import glob
import numpy
import matplotlib
import matplotlib.pyplot as plt
import pickle
import pandas as pd
%matplotlib inline
matplotlib.rcParams.update({'axes.linewidth': 0.25,
                            'xtick.major.size': 2,
                            'xtick.major.width': 0.25,
                            'ytick.major.size': 2,
                            'ytick.major.width': 0.25,
                            'pdf.fonttype': 42,
                            'font.sans-serif': 'Arial'})

In [2]:
def read_array_metadata(metadata,filename,n_levels, extraplot=None):
    array_metadata = metadata[metadata['Count file'] == filename]
    array_levels = [array_metadata['Level %d'%(idx+1)].values[0] for idx in range(0,n_levels)]
    
    if extraplot != None:
        array_levels = array_metadata['Extra_plot_level'].tolist()
    return array_levels

In [3]:
# Load Lambda pmean df
path = '../../smomics_data'

# Read file
filename = os.path.join(path, 'lambdas_pmeans.tsv')

#Read file
#filename = os.path.join(path, 'lambdas_pmeans_spleen_exons_mRNA_ifs.tsv')
 
lambda_posterior_means = pd.read_csv(filename, index_col=0, sep='\t', header=[0,1])

# Take exp()
lambda_posterior_means = lambda_posterior_means.astype(float)
lambda_posterior_means = numpy.exp(lambda_posterior_means-1)

#clean up names
nwe=[]
nm=lambda_posterior_means.index
for item in nm:
    nwe.append(str(item).split("_")[0])
lambda_posterior_means.index = nwe

/Users/svickovi/miniconda3/envs/stenv3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  


# Plot individual sections

In [4]:
# Load sample_information file
# Read file
path = '../../smomics_data'
info_file = os.path.join(path, 'information.p')
info = pickle.load(open(info_file,'rb')) 
    
metadata = info['metadata']
n_levels = info['n_levels']

print(metadata['Count file'][2])
print(n_levels)

FileNotFoundError: [Errno 2] No such file or directory: '../../smomics_data/information.p'

In [5]:
print(metadata['Count file'][1])

NameError: name 'metadata' is not defined

In [10]:
gene = 'Adgre1'

In [ ]:
# initialize a figure
fig = matplotlib.pyplot.figure(figsize=(60, 60))
num_cols = 3
axes = []
count_files = numpy.array(list(lambda_posterior_means.columns.levels[0]))
for i in range(0,int((int(len(count_files)+2)/num_cols))):
    for j in range(0, num_cols):
        ax = plt.subplot2grid((int((len(count_files)+2)/num_cols+1), num_cols), (i, j))
        axes.append(ax) 

# get title
fig.suptitle(gene, fontsize=50)

# set percent cut offs for ploting
mins = [5,5,5,5,5,5]
maxs = [95,95,95,95,95,95]
# loop over the count files
for n, count_file in enumerate(count_files):
    
    ax = axes[n]

    coordinates = numpy.array([list(map(float,coordinate.split('_')))
                               for coordinate in list(lambda_posterior_means[count_file].columns)])
    
    x_coordinates = (coordinates[:,0]-1)
    y_coordinates = (coordinates[:,1]-1)
    
    # visualize the ST spots 
    s=300
    
    vmin = numpy.percentile(lambda_posterior_means[count_file].loc[gene].values,mins[n])
    vmax = numpy.percentile(lambda_posterior_means[count_file].loc[gene].values,maxs[n])
    
    cbmap = ax.scatter(x_coordinates,y_coordinates,
                       s=s,c=lambda_posterior_means[count_file].loc[gene],
                       cmap='magma',vmin=vmin,vmax=vmax,marker='o')
    
    ax.set_aspect('equal')
    
    ax.set_xticks([])
    ax.set_yticks([])
    
    ax.set_xlim(0, 33)
    ax.set_ylim(35,0)
    
    # put some information on the array in the title
    ax.set_title(count_file.split("/")[-1].split("_stdata_adjusted.tsv")[0], fontsize=50)
    
    cbar = matplotlib.pyplot.colorbar(cbmap,ax=ax,shrink=0.4)
    cbar.ax.tick_params(labelsize=50) 
    
    cbar.set_label(r'%s $\bar{\lambda}$'%(gene), fontsize=50)
    
fig.subplots_adjust(top=0.90)
#fig.savefig(gene+'_exons.pdf')

In [117]:
len(count_files)

4